# Usage Examples for `MDDC` in Python

Authors: Raktim Mukhopadhyay, Anran Liu, Marianthi Markatou

In [ ]:
import warnings
from datetime import datetime

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

warnings.filterwarnings("ignore")


current_date = datetime.now().strftime("%d %B, %Y")
display(Markdown(f"**Date Updated:** {current_date}"))

In [ ]:
import MDDC

MDDC.__version__

## Introduction

This vignette contains various examples that illustrate usage of `MDDC`.

### Installation

The `MDDC` package is available on PyPI and can be installed using `pip`. Additionally, the development version can be found on [GitHub](https://github.com/rmj3197/MDDC). 

## Dataset

We have included an adverse event dataset curated from FDA Adverse Event Reporting System (FAERS) called `statin49` dataset which we will be using for describing the functionalities of `MDDC`. `statin49` was downloaded and processed from the FAERS database, covering the period from the third quarter of 2014 (Q3 2014) to the fourth quarter of 2020 (Q4 2020). This dataset is a $50 \times 7$ contingency table. The first 49 rows represent 49 important AEs associated with the statin class, while the final row aggregates the remaining 5,990 AEs. 

In [ ]:
statin49 = MDDC.datasets.load_statin49_data()

In [ ]:
statin49.head()

In [ ]:
statin49_cluster_idx = MDDC.datasets.load_statin49_cluster_idx_data()

In [ ]:
statin49_cluster_idx.head()

## Adverse Event (AE) identification with MDDC

### Using Boxplot Method 

In [ ]:
boxplot_results = MDDC.MDDC.mddc(
    contin_table=statin49,
    method="boxplot",
    col_specific_cutoff=True,
    separate=True,
    if_col_corr=False,
    corr_lim=0.8,
)

In [ ]:
print(
    f"The boxplot method result contains the following components:{boxplot_results._fields}"
)

MDDC with boplot method function outputs a list with three components as seen above:

- `signal`: An $I \times J$ data matrix with entries 1 or 0, indicating the signals identified in step 2. A value of 1 indicates signals, 0 indicates no signal.
- `corr_signal_pval`: An $I \times J$ data matrix of p-values for each cell in the contingency table from step 5, when the $r_{ij}$ values are mapped back to the standard normal distribution.
- `corr_signal_adj_pval`: An $I \times J$ data matrix of the Benjamini-Hochberg adjusted p-values for each cell in step 5. Users can choose whether to use `corr_signal_pval` or `corr_signal_adj_pval`, and can set their own p-value threshold (for example, 0.05).


Below, we display the first few rows and columns for each component of `boxplot_result`. 

In [ ]:
boxplot_results.signal.head(6)

In [ ]:
boxplot_results.corr_signal_pval.round(3).head(6)

In this output, we observe that the first row, corresponding to the adverse event *Rhabdomyolysis*, does not have associated p-values. This is because, in step 2 of the algorithm, *Rhabdomyolysis* was already identified as an AE signal for Atorvastatin, Pravastatin, Rosuvastatin, and Simvastatin. Consequently, the standardized Pearson residual values for these four drugs were replaced with NA. With only two residual values remaining in the first row, it was not possible to find connected AEs for *Rhabdomyolysis*. Therefore, this adverse event was excluded from the subsequent steps of the analysis. 

Note that for computing Pearson correlation in step 3, at least three values are required in the matching positions. Applying a p-value threshold of 0.05, we identify the following pairs as signals by considering AE correlations: 

- (Muscle Rupture, Fluvastatin)
- (Muscle Rupture, Pravastatin)
- (Muscle Disorder, Rosuvastatin)

#### Function for Finding Optimal Boxplot Coefficient

In [ ]:
MDDC.MDDC.find_optimal_coef(
    contin_table=statin49,
    rep=1000,
    target_fdr=0.05,
    grid=0.1,
    col_specific_cutoff=True,
    exclude_small_count=True,
)

This function outputs a list with the following components:

- `coef`: A numeric vector containing the optimal coefficient ‘coef’ for each column of the input contingency table.

- `FDR`: A numeric vector with the corresponding false discovery rate (FDR) for each column.

### Using Monte Carlo Method

In [ ]:
mc_results = MDDC.MDDC.mddc(
    contin_table=statin49,
    method="monte_carlo",
    rep=10000,
    exclude_same_drug_class=True,
    col_specific_cutoff=True,
    separate=True,
    if_col_corr=False,
    corr_lim=0.8,
)

In [ ]:
print(
    f"The Monte Carlo method result contains the following components:{mc_results._fields}"
)

- `pval`: p-values for each cell in the step 2 of the algorithm, calculated using the Monte Carlo method for cells with count greater than five, and Fisher’s exact test for cells with count less than or equal to five. Please note the p-values are not adjusted for multiple testing.
- `pval_adj`: Matrix of adjusted p-values using the Benjamini-Hochberg procedure for each cell in the second step.
- `signal:`: Matrix indicating significant signals with count greater than five and identified in the step 2 by the Monte Carlo method. 1 indicates a signal, 0 indicates non-signal. Please note the unadjusted p-values are used to identify signals here.
- `fisher_signal`: Matrix indicating signals with a count less than or equal to five and identified by Fisher’s exact test. 1 indicates a signal, 0 indicates non-signal. Please note the unadjusted p-values are used to identify signals here.
- `signal_adj`: Matrix indicating significant signals with count greater than five and identified in the step 2 by the Monte Carlo method. 1 indicates a signal, 0 indicates non-signal. Please note the adjusted p-values are used to identify signals here.
- `fisher_signal_adj`: Matrix indicating signals with a count less than or equal to five and identified by Fisher’s exact test. 1 indicates a signal, 0 indicates non-signal. Please note the adjusted p-values are used to identify signals here.
- `corr_signal_pval`: Returns the p-values for each cell in the contingency table in step 5, where the $r_{ij}$ values are mapped back to the standard normal distribution.
- `corr_signal_adj_pval`: Returns the Benjamini-Hochberg adjusted p-values for each cell in step 5. Users can choose whether to use `corr_signal_pval` or `corr_signal_adj_pval`, and select an appropriate p-value threshold (for example, 0.05).


## Function for Reporting 

For ease of use and understanding the identified signals, we have also included a utility function which extracts the signals from
the signal matrix and constructs a dataframe. 

The function `report_drug_AE_pairs()` displays the identified (AE, drug) pairs as well as the observed count, expected count, and the standardized Pearson residuals for the pairs. This function takes two arguments:

- `contin_table`: A data matrix representing an $I \times J$ contingency table, with rows corresponding to adverse events and columns corresponding to drugs.
- `contin_table_signal`: A data matrix with the same dimensions and row and column names as `contin_table`. Entries should be either 1 (indicating a signal) or 0 (indicating no signal). This matrix can be obtained by applying the `MDDC.MDDC.mddc()` functions to `contin_table`.


In [ ]:
report_df = MDDC.utils.report_drug_AE_pairs(statin49, mc_results.signal_adj)

report_df.head()

## Simulating datasets with grouped AEs

- `contin_table`: A data matrix representing an $I \times J$ contingency table with rows corresponding to adverse events and columns corresponding to drugs. The row and column marginals are used to generate the simulated data.
- `signal_mat`: A data matrix of the same dimensions as the contingency table with entries indicating the signal strength. Values must be greater than or equal to 1, where 1 indicates no signal, and values greater than 1 indicate a signal.
- `cluster_idx`: A numpy.ndarray, list or pandas.DataFrame denoting the cluster index of the various AEs. 
- `n`: The number of simulated contingency tables to be generated.
- `rho`: A numeric value representing the correlation of the AEs within each cluster. The default is 0.5.
- `n_job:`: n_jobs specifies the maximum number of concurrently running workers.
- `seed`:  Random seed for reproducibility of the simulation.

In [ ]:
contin_table = statin49_cluster_idx.drop(columns=["cluster_idx"])
cluster_idx = statin49_cluster_idx["cluster_idx"].to_list()
signal_mat = pd.DataFrame(np.ones(contin_table.shape))
signal_mat.columns = list(contin_table.columns)
signal_mat.index = list(contin_table.index)

# A signal of strength 4 is embedded in Rhabdomyolysis for Atorvastatin
signal_mat.at["Rhabdomyolysis", "Atorvastatin"] = 4

In [ ]:
simulated_datasets = MDDC.utils.generate_contin_table_with_clustered_AE(
    row_marginal=None,
    column_marginal=None,
    contin_table=contin_table,
    signal_mat=signal_mat,
    cluster_idx=cluster_idx,
    n=3,
    rho=0.5,
    seed=42,
)

In [ ]:
mc_results_with_sim_data = MDDC.MDDC.mddc(simulated_datasets[0], method="monte_carlo")

In [ ]:
mc_report_with_sim_datasets_df = MDDC.utils.report_drug_AE_pairs(
    simulated_datasets[0], mc_results_with_sim_data.signal
)

mc_report_with_sim_datasets_df

In the above output, we can see that the identified signal pair is (Atorvastatin, Rhabdomyolysis) which matches the signal embedded previously.

## Visualization

We have also included heatmap visulizations as a part of our package to help visualize the identified signals or p-values. 

This function takes the following arguments: 
- `data:` The data to be plotted as a heatmap. This attribute should be a numpy.ndarray or pandas.DataFrame.
- `size_cell:` The size of each cell in the heatmap, which affects the dimensions of the resulting plot.

In [ ]:
# visualizing the identified signals
MDDC.utils.plot_heatmap(mc_results.signal_adj.iloc[:-1], cmap="Blues")

In [ ]:
# visualizing the p-values
MDDC.utils.plot_heatmap(mc_results.pval_adj.iloc[:-1], cmap="Blues_r")